# Lab 3 — clustering
**Internet Analytics - Lab 3**

---

**Group:** *K*

**Names:**

* *Robin Lang*
* *Kim Lan Phan Hoang*

In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pickle
import random

import sklearn.cluster as scl

%matplotlib inline
plt.style.use("ggplot")

## Exercice 3.11

### Cluster the data using the k-means clustering algorithm.

In [ ]:
#http://scikit-learn.org/stable/modules/generated/sklearn.cluster.k_means.html

In [ ]:
km2 = scl.k_means(data,2)

In [ ]:
km5 = scl.k_means(data,5)

### Visualize the clusters using an interactive bokeh plot. Try projecting on different principal directions

### Which principal directions separate the clusters well?

1.5

## Exercice 3.12

### Create a dict that maps movie IDs to set of genres from the data in movies.txt

In [6]:
movies = sc.textFile("/ix/ml-20m/movies.txt").map(json.loads).collect()

In [12]:
idToGenre = {}
for x in movies:
    idToGenre[x["movieId"]] = x["genres"]

### Implement the k-medioids algorithm with the Jaccard distance.

In [14]:
def intersect(a, b):
     return list(set(a) & set(b))
    
def union(a,b):
    return set().union(a,b)

def jaccard_index(a,b):
    return len(union(a,b)) / len(intersect(a,b))

def jaccard_distance(a,b):
    return 1-jaccard_index(a,b)

In [15]:
def kmedioids(points, k):
    medioids = np.array(k)
    clusters = np.array(k)
    
    # initialize medioids
    for i in range(k):
        medioids[i] = random.choice(points)
        clusters[i] = []
        
    # convergence parameters
    convergedLimit = 2
    convergedLoop = 0
        
    # repeat until convergence
    while(convergedLoop < convergedLimit):
        
        #initialize cluster_tmp (use it to check convergence)
        clusters_tmp = np.array(k)
        for i in range(k):
            clusters_tmp[i] = []
            
        
        # class points in clusters
        for p in points:
            min_i = 0
            jdk = 1000000000
            for i in range(k):
                jdi = jaccard_distance(idToGenre[medioids[i]], idToGenre[p])
                if(jdi < jdk):
                    min_i = i
                    jdk = jdi
                    
            clusters_tmp[min_i].append(p)
        
        #recompute medioids -> choose the closest point to the others
        for i in range(k):
            best_x = 0
            for x in clusters_tmp[i]:
                
                add = 0
                best_add = 1000000000
                
                for y in clusters_tmp[i]:
                    add = add + jaccard_distance(idToGenre[x],idToGenre[y])
                    
                if add < best_add:
                    best_add = add
                    best_x = x
                    
            medioids[i] = best_x
            
        # check convergence
        if(clusters_tmp == clusters):
            convergedLoop = convergedLoop + 1
        else :
            convergedLoop = 0

### Cluster the set of movies in the file most-rated.pickle, using k = 2

In [21]:
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, ColumnDataSource
from bokeh.models import HoverTool, ResetTool, PanTool, WheelZoomTool, SaveTool
output_notebook()
import pickle

# Reading an object from disk.
with open("selected-movies.pickle", "rb") as f:
    f = pickle.load(f, encoding="utf-8")
    

for movie in movies:
    print(movie)

Loading BokehJS ...

UnpicklingError: the STRING opcode argument must be quoted

In [ ]:
kmedioids(2)

### Find a good way to visualize the results of the clustering. How do you intepret the two clusters?

[1, 2]